<h1 align="center"> Deep Learning gender from name - RNN LSTMs </h1>

#### we will use an LSTM RNN to learn gender as f(name). we will use a stacked LSTM with many-to-one architecture feeding charecter inputs and predicting a binary outcome M/F. loss function used will be binary_crossentropy (a special case of categorical_crossentropy with m=2) and using adam optimizer (modified SGD) sample input /output would like this <br> ['r','a','k','e','s','h',' '] - male<br> ['p','r','a','d','e','e','p'] - male<br> ['g','a','n','g','a',' '] - female<br> and so on...

<img src="LSTM_RNN_architecture.jpg" width="800" height="600"/>

In [1]:
from __future__ import print_function

from sklearn.preprocessing import OneHotEncoder
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
import pandas as pd
import numpy as np
import os

/Users/robertpatterson/.virtualenvs/keras_tf_py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#parameters
maxlen = 30
labels = 2

In [3]:
input = pd.read_csv("gender_data.csv",header=None)
input.columns = ['name','m_or_f']
input['namelen']= [len(str(i)) for i in input['name']]
input1 = input[(input['namelen'] >= 2) ]

In [4]:
input1.groupby('m_or_f')['name'].count()

m_or_f
f    6705
m    8475
Name: name, dtype: int64

In [5]:
names = input['name']
gender = input['m_or_f']
vocab = set(' '.join([str(i) for i in names]))
vocab.add('END')
len_vocab = len(vocab)

In [6]:
print(vocab)
print("vocab length is ",len_vocab)
print ("length of input is ",len(input1))

{'z', 'c', 'p', 'END', 'd', '3', 'i', 'o', '4', '.', 'w', 'n', '1', 'f', 'l', '0', 'a', 'v', 'e', '9', 't', 'y', 'r', 'g', ' ', 'b', 'u', 's', 'j', '8', 'q', 'm', 'x', '7', '2', 'k', '6', 'h', '5'}
vocab length is  39
length of input is  15226


In [7]:
char_index = dict((c, i) for i, c in enumerate(vocab))

In [8]:
print(char_index)

{'z': 0, 'c': 1, 'p': 2, 'END': 3, 'd': 4, '3': 5, 'i': 6, 'o': 7, '4': 8, '.': 9, 'w': 10, 'n': 11, '1': 12, 'f': 13, 'l': 14, '0': 15, 'a': 16, 'v': 17, 'e': 18, '9': 19, 't': 20, 'y': 21, 'r': 22, 'g': 23, ' ': 24, 'b': 25, 'u': 26, 's': 27, 'j': 28, '8': 29, 'q': 30, 'm': 31, 'x': 32, '7': 33, '2': 34, 'k': 35, '6': 36, 'h': 37, '5': 38}


In [9]:
#train test split
msk = np.random.rand(len(input1)) < 0.8
train = input1[msk]
test = input1[~msk]     

In [10]:
#take input upto max and truncate rest
#encode to vector space(one hot encoding)
#padd 'END' to shorter sequences
train_X = []
trunc_train_name = [str(i)[0:30] for i in train.name]
for i in trunc_train_name:
    tmp = [char_index[j] for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(char_index["END"])
    train_X.append(tmp)

In [11]:
np.asarray(train_X).shape

(12223, 30)

In [12]:
def set_flag(i):
    tmp = np.zeros(39);
    tmp[i] = 1
    return(tmp)

In [13]:
set_flag(3)

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.])

#### modify the code above to also convert each index to one-hot encoded representation

In [14]:
#take input upto max and truncate rest
#encode to vector space(one hot encoding)
#padd 'END' to shorter sequences
#also convert each index to one-hot encoding
train_X = []
train_Y = []
trunc_train_name = [str(i)[0:maxlen] for i in train.name]
for i in trunc_train_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    train_X.append(tmp)
for i in train.m_or_f:
    if i == 'm':
        train_Y.append([1,0])
    else:
        train_Y.append([0,1])
    

In [15]:
np.asarray(train_X).shape

(12223, 30, 39)

In [16]:
np.asarray(train_Y).shape

(12223, 2)

#### build model in keras ( a stacked LSTM model with many-to-one arch ) here 30 sequence and 2 output each for one category(m/f)

In [18]:
#build the model: 2 stacked LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen,len_vocab)))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 30, 512)           1130496   
_________________________________________________________________
dropout_3 (Dropout)          (None, 30, 512)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1026      
_________________________________________________________________
activation_2 (Activation)    (None, 2)                 0         
Total params: 3,230,722
Trainable params: 3,230,722
Non-trainable params: 0
___________________________________________________

In [19]:
test_X = []
test_Y = []
trunc_test_name = [str(i)[0:maxlen] for i in test.name]
for i in trunc_test_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    test_X.append(tmp)
for i in test.m_or_f:
    if i == 'm':
        test_Y.append([1,0])
    else:
        test_Y.append([0,1])
    

In [20]:
print(np.asarray(test_X).shape)
print(np.asarray(test_Y).shape)

(3003, 30, 39)
(3003, 2)


In [25]:
batch_size=1000
model.fit(np.asarray(train_X), np.asarray(train_Y),batch_size=batch_size,epochs=10,validation_data=(np.asarray(test_X), np.asarray(test_Y)))

Train on 12223 samples, validate on 3003 samples
Epoch 1/10
12223/12223 [==============================] - 74s 6ms/step - loss: 0.6938 - acc: 0.5502 - val_loss: 0.6858 - val_acc: 0.5564
Epoch 2/10
12223/12223 [==============================] - 71s 6ms/step - loss: 0.6750 - acc: 0.5742 - val_loss: 0.6436 - val_acc: 0.6277
Epoch 3/10
12223/12223 [==============================] - 72s 6ms/step - loss: 0.6110 - acc: 0.6698 - val_loss: 0.5854 - val_acc: 0.6996
Epoch 4/10
12223/12223 [==============================] - 71s 6ms/step - loss: 0.5726 - acc: 0.7031 - val_loss: 0.5499 - val_acc: 0.7299
Epoch 5/10
12223/12223 [==============================] - 68s 6ms/step - loss: 0.5425 - acc: 0.7344 - val_loss: 0.5179 - val_acc: 0.7609
Epoch 6/10
12223/12223 [==============================] - 67s 6ms/step - loss: 0.5082 - acc: 0.7528 - val_loss: 0.5047 - val_acc: 0.7609
Epoch 7/10
12223/12223 [==============================] - 67s 5ms/step - loss: 0.4977 - acc: 0.7646 - val_loss: 0.4968 - val_acc:

In [27]:
score, acc = model.evaluate(np.asarray(test_X), np.asarray(test_Y))
print('Test score:', score)
print('Test accuracy:', acc)

3003/3003 [==============================] - 9s 3ms/step
Test score: 0.4898969696693884
Test accuracy: 0.7835497834703901


In [30]:
name=["sandhya","jaspreet","rajesh"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))

In [31]:
pred

array([[0.62659043, 0.37340954],
       [0.5595104 , 0.4404896 ],
       [0.7332715 , 0.26672855]], dtype=float32)

#### Lets train more, clearly some very simple female names it doesnt get right like mentioned above (inspite it exists in training data)

In [ ]:
batch_size=1000
model.fit(np.asarray(train_X), np.asarray(train_Y),batch_size=batch_size,epochs=50,validation_data=(np.asarray(test_X), np.asarray(test_Y)))

Train on 12223 samples, validate on 3003 samples
Epoch 1/50
12223/12223 [==============================] - 71s 6ms/step - loss: 0.4775 - acc: 0.7747 - val_loss: 0.4700 - val_acc: 0.7902
Epoch 2/50
12223/12223 [==============================] - 68s 6ms/step - loss: 0.4621 - acc: 0.7840 - val_loss: 0.4719 - val_acc: 0.7919
Epoch 3/50
12223/12223 [==============================] - 69s 6ms/step - loss: 0.4586 - acc: 0.7839 - val_loss: 0.4516 - val_acc: 0.7969
Epoch 4/50
12223/12223 [==============================] - 67s 6ms/step - loss: 0.4482 - acc: 0.7948 - val_loss: 0.4519 - val_acc: 0.8045
Epoch 5/50
12223/12223 [==============================] - 68s 6ms/step - loss: 0.4385 - acc: 0.8000 - val_loss: 0.4419 - val_acc: 0.7969
Epoch 6/50
12223/12223 [==============================] - 67s 5ms/step - loss: 0.4242 - acc: 0.8095 - val_loss: 0.4536 - val_acc: 0.7975
Epoch 7/50
12223/12223 [==============================] - 68s 6ms/step - loss: 0.4171 - acc: 0.8133 - val_loss: 0.4331 - val_acc:

In [460]:
score, acc = model.evaluate(test_X, test_Y)
print('Test score:', score)
print('Test accuracy:', acc)

3028/3028 [==============================] - 16s    
Test score: 0.448404541104
Test accuracy: 0.864266842879


<h3 align="center"> lets look at the loss and accuracy chart as a function of epochs </h3><img src="loss_charts.bmp" alt="loss charts" width="500" height="350"/><img src="acc_charts.bmp" alt="loss charts"  width="500" height="350"/>

In [342]:
name=["sandhya","jaspreet","rajesh","kaveri","aditi deepak","arihant","sasikala","aditi","ragini rajaram"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))
pred

array([[ 0.0859881 ,  0.91401184],
       [ 0.96310365,  0.03689628],
       [ 0.7148453 ,  0.28515476],
       [ 0.02246205,  0.97753793],
       [ 0.13607673,  0.86392319],
       [ 0.99559009,  0.00440993],
       [ 0.05380283,  0.94619709],
       [ 0.55060732,  0.44939268],
       [ 0.10676169,  0.89323831]], dtype=float32)

In [345]:
name=["abhi","abhi deepak","mr. abhi"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))
pred

array([[ 0.15557961,  0.84442037],
       [ 0.25342518,  0.74657482],
       [ 0.8618474 ,  0.13815261]], dtype=float32)

In [502]:
name=["rajini","rajinikanth","mr. rajini"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))
pred

array([[ 0.33718896,  0.66281104],
       [ 0.99896383,  0.00103616],
       [ 0.99664474,  0.00335527]], dtype=float32)

In [450]:
#save our model and data
model.save_weights('gender_model',overwrite=True)
train.to_csv("train_split.csv")
test.to_csv("test_split.csv")

In [464]:
evals = model.predict(test_X)
prob_m = [i[0] for i in evals]

In [479]:
out = pd.DataFrame(prob_m)
out['name'] = test.name.reset_index()['name']
out['m_or_f']=test.m_or_f.reset_index()['m_or_f']

In [483]:
out.head(10)
out.columns = ['prob_m','name','actual']
out.head(10)
out.to_csv("gender_pred_out.csv")